In [22]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [8]:
df = pd.read_csv('apollo_data.csv')
df.head()

,Unnamed: 0,age,sex,smoker,region,viral load,severity level,hospitalization charges
0,0,19,female,yes,southwest,9.30,0,42212
1,1,18,male,no,southeast,11.26,1,4314
2,2,28,male,no,southeast,11.00,3,11124
3,3,33,male,no,northwest,7.57,0,54961
4,4,32,male,no,northwest,9.63,0,9667


In [9]:
# drop unnecessary column 1
df.drop(columns='Unnamed: 0', inplace= True)
df.head()

,age,sex,smoker,region,viral load,severity level,hospitalization charges
0,19,female,yes,southwest,9.30,0,42212
1,18,male,no,southeast,11.26,1,4314
2,28,male,no,southeast,11.00,3,11124
3,33,male,no,northwest,7.57,0,54961
4,32,male,no,northwest,9.63,0,9667


In [10]:
# data cleaning process starts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      1338 non-null   int64  
 1   sex                      1338 non-null   object 
 2   smoker                   1338 non-null   object 
 3   region                   1338 non-null   object 
 4   viral load               1338 non-null   float64
 5   severity level           1338 non-null   int64  
 6   hospitalization charges  1338 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 73.3+ KB


In [11]:
df.describe()

,age,viral load,severity level,hospitalization charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,10.221233,1.094918,33176.058296
std,14.049960,2.032796,1.205493,30275.029296
min,18.000000,5.320000,0.000000,2805.000000
25%,27.000000,8.762500,0.000000,11851.000000
50%,39.000000,10.130000,1.000000,23455.000000
75%,51.000000,11.567500,2.000000,41599.500000
max,64.000000,17.710000,5.000000,159426.000000


In [12]:
# check for null values
df.isna().sum()

age                        0
sex                        0
smoker                     0
region                     0
viral load                 0
severity level             0
hospitalization charges    0
dtype: int64

In [14]:
# business problem no 1
# we first need to get dummies

df['sex'].unique()

array(['female', 'male'], dtype=object)

In [15]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [16]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [17]:
df_dummies = pd.get_dummies(df, columns=['sex', 'smoker', 'region'])
df_dummies.head()

,age,viral load,severity level,hospitalization charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,9.30,0,42212,True,False,False,True,False,False,False,True
1,18,11.26,1,4314,False,True,True,False,False,False,True,False
2,28,11.00,3,11124,False,True,True,False,False,False,True,False
3,33,7.57,0,54961,False,True,True,False,False,True,False,False
4,32,9.63,0,9667,False,True,True,False,False,True,False,False


In [19]:
# we will now change these into 0 or 1

df_dummies.columns

Index(['age', 'viral load', 'severity level', 'hospitalization charges',
       'sex_female', 'sex_male', 'smoker_no', 'smoker_yes', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest'],
      dtype='object')

In [20]:
dummy_columns = ['sex_female', 'sex_male', 'smoker_no', 'smoker_yes', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest']

for col in dummy_columns:
    df_dummies[col] = df_dummies[col].astype(int)

df_dummies.head()

,age,viral load,severity level,hospitalization charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,9.30,0,42212,1,0,0,1,0,0,0,1
1,18,11.26,1,4314,0,1,1,0,0,0,1,0
2,28,11.00,3,11124,0,1,1,0,0,0,1,0
3,33,7.57,0,54961,0,1,1,0,0,1,0,0
4,32,9.63,0,9667,0,1,1,0,0,1,0,0


In [23]:
# fitting OLS

X = sm.add_constant(df_dummies[['age', 'viral load', 'severity level',
       'sex_female', 'sex_male', 'smoker_no', 'smoker_yes', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest']])
y = df_dummies['hospitalization charges']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                              
Dep. Variable:     hospitalization charges   R-squared:                       0.751
Model:                                 OLS   Adj. R-squared:                  0.749
Method:                      Least Squares   F-statistic:                     500.9
Date:                     Thu, 19 Dec 2024   Prob (F-statistic):               0.00
Time:                             23:31:51   Log-Likelihood:                -14774.
No. Observations:                     1338   AIC:                         2.957e+04
Df Residuals:                         1329   BIC:                         2.961e+04
Df Model:                                8                                         
Covariance Type:                 nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

> 1. Model Summary

R-squared: 0.751 means 75.1% of the variance in hospitalization charges is explained by the model.
F-statistic: 500.9 with the P-value converging to 0.

> 2. Coefficient

Higher absolute coefficient means a stronger significance level to the target variable.
The patient being a smoker or not has the strongest significance in deciding the hospitalization charges. smoker_no has a coef of -30180, meaning if the patient is a non smoker the charge of the hospitalization will drop by $30180
Being a smoker will increase the charge to $29440

> 3. Statistical Significance

>>sex variables both have a high p-value more than the significance level 0.05; sex_female 0.758, sex_male 0.436 meaning the variables are not significantly correlated to the target variable.
>>region also seems to have a lower significance; all region related variable have p-value level higher than 0.05.
>>smoker variables both have the p-value converging to 0 which is smaller than 0.05. We can say smoker variable is has the highest significance in predicting the reason for hospitalization charge.
>>viral load and severity level also have a strong significance to the target variable. Also the p-value for both variables are less than 0.05.